# Central Exercise 8 - Joukowski transform - Flow around an airfoil

This script plots streamlines and potential lines for the flow past an airfoil. Also, several plots to visualize the Joukowski transform are provided. The flow around an airfoil results from superimposing a dipole and a parallel flow in the z_prime plane, which results in the flow past a cylinder in the z_prime plane. To calculate the flow past an airfoil, a Joukowski transformation (a conformal mapping) is used. To enforce the Kutta condition, a potential vortex is superimposed to the cylinder flow in the z_prime plane.

## Joukowski transform

To calculate the flow past an airfoil a Joukowski transform is required. It is implemented as class in the next cell. It provides three main functionalities: A member function to map from the z_prime to the z plane (forward transform), a member function to transform from the z to the z_prime plane (inverse transform), and a member function to calculate the derivative of the forward transform with respect to z_prime. This is necessary to calculate correct velocities in the z plane.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class JoukowskiTranformation:

    def __init__(self, center: np.complex128, c_constant: float = 1.0) -> None:
        """Constructor for the Joukowski transform class. Initializes constants that determine the shape of the transformed object.

        Args:
            center (np.complex128): The center of the circle in the z_prime plane.
            c_constant (float, optional): A constant that influences the Joukowski transform. Defaults to 1.0.
        """
        self.center = center
        self.c_constant = c_constant
        self.radius = np.absolute( 1.0 - self.center )

    def transform(self, z_prime: np.array) -> np.array:
        """The forward Joukowski transformation. It transforms the z_prime plane (the "simple" plane, where we usually consider the circle/cylinder)
        to the z plane (the "interesting" plane, where we have the flow past the transformed object, which is usually a foil).

        Args:
            z_prime (np.array): The complex numpy array that provides the coordinates in the z_prime plane.

        Returns:
            np.array: The complex numpy array that provides the coordinates in the z plane.
        """
        z = z_prime
        z = z + self.center
        return z + self.c_constant**2 / z
    
    def inverse_transform(self, z: np.array) -> np.array:
        """The inverse of the Joukowski transformation. It transforms the z plane (the "interesting" plane, where we have the flow past the transformed object, which is usually a foil)
        to the z_prime plane (the "simple" plane, where we usually consider the circle/cylinder).

        Args:
            z (np.array): The complex numpy array that provides the coordinates in the z plane.

        Returns:
            np.array: The complex numpy array that provides the coordinates in the z_prime plane.
        """
        r1 = 0.5 * ( z + np.sqrt(z**2 - 4 * self.c_constant**2) )
        r2 = 0.5 * ( z - np.sqrt(z**2 - 4 * self.c_constant**2) )

        r1 = r1 - self.center
        r2 = r2 - self.center

        z_prime = np.where(np.absolute(r1) > np.absolute(r2), r1, r2)

        return z_prime

    def derivative(self, z_prime: np.array) -> np.array:
        """Returns the derivative of the Joukowski with respect to z_prime. This is necessary to calculate correct velocities in the z plane.

        Args:
            z_prime (np.array): The coordinates in the z_prime plane for which the derivative should be calculated as a complex numpy array.

        Returns:
            np.array: The derivative at the specified locations as a numpy array-
        """
        z = z_prime
        z = z + self.center
        derivative = 1 - self.c_constant**2 / z**2
        return derivative


## Preparation

To prepare the calculations, several helper functions have to be defined. This is done in the following cell.
First, the two libraries numpy and matplotlib.pyplot have to be imported in order to user their functionality. Note, these libraries are helpful in many scenarios, when data should be plotted. Next, helper classes that provide the complex velocity for the source-/sink-flow and the parallel flow are implemented. Finally, a function providing the meshgrid for the calculations is implemented.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class ParallelFlow:

    def __init__(self, velocity_x: float, velocity_y: float) -> None:
        """Initializer for the parallel flow class. Stores relevant quantities as class members.

        Args:
            velocity_x (float): Cartesion component of the velocity pointing in x-direction.
            velocity_y (float): Cartesion component of the velocity pointing in y-direction.
        """
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Return the complex velocity of the parallel flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        return self.velocity_x - 1.0j * self.velocity_y

class DipolFlow:

    def __init__(self, dipol_strength, translation_x = 0.0, translation_y = 0.0) -> None:
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.dipol_strength = dipol_strength
        self.factor = self.dipol_strength / np.pi

    def get_complex_velocity(self, z):
        z = z - self.translation_x - 1.0j * self.translation_y
        return - self.factor / (z*z)

class PotentialVortexFlow:

    def __init__(self, gamma: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the vortex flow class. Stores relevant quantities as class members.

        Args:
            gamma (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.gamma = gamma
        self.factor = self.gamma / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the vortex flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z = z - self.translation_x - 1.0j * self.translation_y
        return - 1.0j * self.factor / z

class SourceFlow:

    def __init__(self, source_strength: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the source- and sink-flow class. Stores relevant quantities as class members.

        Args:
            source_strength (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.source_strength = source_strength
        self.factor = self.source_strength / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the source-/sink-flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z = z - self.translation_x - 1.0j * self.translation_y
        return self.factor / z
    
def get_grid_quantities(x_min: float, x_max: float, y_min: float, y_max: float, resolution_x: int, resolution_y: int):
    """Returns the meshgrid used for the calculations.

    Args:
        x_min (float): The minimum x value.
        x_max (float): The maximum x value.
        y_min (float): The minimum y value.
        y_max (float): The maximum y value.
        resolution_x (int): The number of points to discretize in x direction.
        resolution_y (int): The number of points to discretize in y direction.

    Returns:
        List[np.array]: The numpy array containing the meshgrid.
    """
    x_linspace = np.linspace(x_min, x_max, resolution_x)
    y_linspace = np.linspace(y_min, y_max, resolution_y)
    X,Y = np.meshgrid(x_linspace, y_linspace)
    Z = X + 1.0j * Y
    return X, Y, Z

## Definition of domain

Here, relevant information to describe the domain is provided. It is used to generate the meshgrid for the calculations.

In [ ]:
x_min = -3.0
x_max = 3.0
y_min = -3.0
y_max = 3.0
resolution_per_unit_length = 20
resolution_x = ( x_max - x_min ) * resolution_per_unit_length
resolution_y = ( y_max - y_min ) * resolution_per_unit_length

X, Y, Z = get_grid_quantities(x_min, x_max, y_min, y_max, int(resolution_x), int(resolution_y))

## Definition of important quantities characterizing the flow

In the following cell, several quantities that specify the flow past the airfoil are specified.

First, we define the origin of the circle in the z_prime plane, which specifies the shape of the foil. Note, it consists of real and imaginary part. The origin of the circle is thus shifted in the x- and y-direction.

In case the shift in x-direction is zero, the foil is a flat plat. A non-zero shift in x-direction adjusts the drop-like shape of the profile.

In case the shift in y-direction is zero, the foil is symmetric with respect to the y-axis. A non-zero shift in y-directions adjust the camber of the profile.

Also, the freestream velocity vector is defined by its components U_infty and V_infty. For V_infty=0, the angle-of-attack is zero.

In [ ]:
center = -0.1 + 0.2j
U_infty = 1.0
V_infty = 0.1

beta = np.arctan2(U_infty, V_infty)
print(f"Angle of the freestream velocity vector, which corresponds to the angle-of-attack: {(90 - np.rad2deg(beta))}")
total_inflow_velocity = np.sqrt(U_infty**2 + V_infty**2)
print(f"Absolute value of the inflow velocity:                                             {total_inflow_velocity}")

## Initialization and first visualizations of the Joukowski transform

We initialize the Joukowski transform and investigate the resulting airfoil.

In [ ]:
joukowski_transformation = JoukowskiTranformation(center)

phi_linspace = np.linspace(0.0, 2.0*np.pi, 100)
radius = joukowski_transformation.radius
print(f"Radius: {radius}")

x_circle = radius * np.cos(phi_linspace)
y_circle = radius * np.sin(phi_linspace)
z_circle = x_circle + 1.0j * y_circle

z_prime = z_circle

z_foil = joukowski_transformation.transform(z_prime)
z_retransformed = joukowski_transformation.inverse_transform(z_foil)

plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

plt.scatter(np.real(z_prime), np.imag(z_prime), c="red")
plt.scatter(np.real(z_foil), np.imag(z_foil), c="green")
plt.scatter(np.real(z_retransformed), np.imag(z_retransformed), c="blue", marker="x")

ax.legend(
    ["Initial circle in z_prime plane", "Airfoil (transformed circle) in the z plane", "retransformed foil, which again gives the circle in the z_prime plane"],
    loc='upper center', bbox_to_anchor=(0.5, 1.3),
)


plt.show()



## What happens with the inner part of the circle

Here, we visualize the transformation of points inside the circle that is mapped to the foil. We see, they are fold to the outside of the circle. This is especially visible when considering the flat plate (center = -0.0 + 0.0j)

In [ ]:
color_list=["red", "green", "magenta", "cyan", "orange", "blue"]

plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

for i in range(6):

    plot_x_circle = (radius - i * radius / 7) * np.cos(phi_linspace)
    plot_y_circle = (radius - i * radius / 7) * np.sin(phi_linspace)
    plot_z_circle = plot_x_circle + 1.0j * plot_y_circle
    plot_z_prime = plot_z_circle
    plot_z_foil = joukowski_transformation.transform(plot_z_prime)


    plt.scatter(np.real(plot_z_prime), np.imag(plot_z_prime), c=color_list[i], s=0.4)
    plt.scatter(np.real(plot_z_foil), np.imag(plot_z_foil), c=color_list[i], s=0.4)

plt.savefig("tranformation.pdf")
plt.show()


## Calculation of deduced quantities

In order to properly impose the conditions specified above (freestream velocity...), several quantities describing the resulting complex potential (dipole moment, circulation to impose Kutta conditions...) have to be derived.

In [ ]:
radius = joukowski_transformation.radius
dipole_strength = np.pi * radius**2 * total_inflow_velocity

small_gamma = - np.imag(joukowski_transformation.center) / radius - np.cos(beta)
gamma = 4.0 * np.pi * total_inflow_velocity * radius * small_gamma

## Calculation of the velocity field in the z plane

To calculate and visualize relevant quantities, we need the complex velocity for a meshgrid in the z_plane. Thus, as usual, we initialize the objects for the elementary flows.

These elementary flows have to be evaluated in the z_prime plane. Thus, we need the inverse transform for the meshgrid in the z plane.

Note, here you can "deactivate" the effect of the potential vortex that enforces the Kutta condition. In case this is done, the flow around the trailing edge can be observed.

In [ ]:
parallel_flow = ParallelFlow(U_infty, V_infty)
dipole_flow = DipolFlow(dipole_strength, 0.0, 0.0)
vortex_flow = PotentialVortexFlow(gamma, 0.0, 0.0)


Z_inverse = joukowski_transformation.inverse_transform(Z)
Z_inverse = np.where(np.absolute(Z_inverse) <= joukowski_transformation.radius, np.nan, Z_inverse)

complex_velocity = parallel_flow.get_complex_velocity(Z_inverse) + dipole_flow.get_complex_velocity(Z_inverse) + vortex_flow.get_complex_velocity(Z_inverse)
complex_velocity = complex_velocity / joukowski_transformation.derivative(Z_inverse)

u = np.real(complex_velocity)
v = - np.imag(complex_velocity)

## Visualization of the results

Finally, we van visualize the results.

### Streamline plot
Plot the streamlines and potential lines.
Streamlines are plotted in TUMBlue.
Potential lines are plotted in TUMOrange.
The contour of the foil is plotted in TUMGreen.

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

x_linspace = np.linspace(x_min, x_max, 21)
y_linspace = np.linspace(x_min, x_max, 50)

# Plot the streamlines.

# The streamline seeds describe all the points where streamlines start. Thus, only streamline the go through these points are drawn.
# This may heavily influence the look of the plot!
# In this case only points to the left (*xmin refers to the left, replace with e.g. 0.0 to also obtain streamlines inside the oval)
# of the oval are choosen. Thus, no streamlines inside the oval show.
# In this case, they are equally distributed along the y direction. To also indicate the stagnation streamlines, values slightly above
# and below (+/- 1.e-5) the x axis are added.
streamline_seeds = np.stack([np.ones_like(y_linspace)*x_min, y_linspace], axis=-1)

ax.streamplot(
    X, Y, u, v,
    density=[1.0, 1.0],
    broken_streamlines=False,
    linewidth=0.6,
    # start_points=streamline_seeds,
    color="#3070b3", # This beautiful color is TUMBlue
)

ax.streamplot(
    X, Y, -v, u,
    density=[0.2, 0.2],
    broken_streamlines=False,
    arrowstyle = "-",
    linewidth=0.6,
    color="#E37222", # This beautiful color is TUMOrange
)

# plt.quiver(X, Y, u, v)

plt.plot(np.real(z_foil), np.imag(z_foil), linewidth=1.6, color="#A2AD00")

# plt.scatter(np.real(z_foil), np.imag(z_foil), c=np.absolute(complex_velocity), s=0.1)
# plt.quiver(np.real(z_foil), np.imag(z_foil), u, v)

plt.savefig("foil.pdf")
plt.show()

### Pressure coefficient

The pressure coefficient can be calculated solely based on the velocity field and the given freestream velocity

In [ ]:
q_squared = np.absolute(complex_velocity)**2
pressure_coefficient = 1.0 - q_squared / total_inflow_velocity**2

plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

im = plt.pcolor(X, Y, pressure_coefficient)

cbar = plt.colorbar(im)
cbar.set_label('Pressure coefficient C_p')

plt.show()

### Pressure distribution along the airoil's surface.

Here, we calculate and plot the pressure coefficient along the surface of the airfoil.

In the plot, blue points indicate points on the upper half of the foil, and red points indicate points on the lower half of the foil.

In [ ]:
phi_linspace = np.linspace(0.0, 2.0*np.pi, 100)
radius = joukowski_transformation.radius
print(f"Radius: {radius}")

x_circle = radius * np.cos(phi_linspace)
y_circle = radius * np.sin(phi_linspace)
z_circle = x_circle + 1.0j * y_circle

z_prime = z_circle

z_foil = joukowski_transformation.transform(z_prime)
x_foil = np.real(z_foil)
y_foil = np.imag(z_foil)

delta_x_foil_left = x_foil - np.roll(x_foil, 1)
side_indicator = np.zeros_like(x_foil)
side_indicator = np.where(delta_x_foil_left > 0, 1, side_indicator)
side_indicator = np.where(delta_x_foil_left < 0, 2, side_indicator)

complex_velocity = parallel_flow.get_complex_velocity(z_prime) + dipole_flow.get_complex_velocity(z_prime) + vortex_flow.get_complex_velocity(z_prime)
complex_velocity = complex_velocity / joukowski_transformation.derivative(z_prime)

u = np.real(complex_velocity)
v = - np.imag(complex_velocity)

q_squared = np.absolute(complex_velocity)**2
pressure_coefficient = 1.0 - q_squared / total_inflow_velocity**2

pressure_coefficient_lower = np.where(side_indicator == 1, pressure_coefficient, np.nan)
pressure_coefficient_upper = np.where(side_indicator == 2, pressure_coefficient, np.nan)

plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

ax.scatter(np.real(z_foil), pressure_coefficient_lower, c="red")
ax.scatter(np.real(z_foil), pressure_coefficient_upper, c="blue")
ax.set_xlabel("x coordinate of point on the foil")
ax.set_ylabel("C_p value")
ax.set_ylim(-3.0, 1.0)

plt.show()